In [ ]:
# Initialize Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnull, count, regexp_extract, split, coalesce, lit, to_date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, ArrayType
import pyspark.sql.functions as F

# Create Spark Session
spark = SparkSession.builder \
    .appName("HighPerformancePySpark") \
    .getOrCreate()

# Set log level to WARN to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

In [ ]:
# Define schema for the dataset
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_details", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("quantity", StringType(), True),
    StructField("price_per_unit", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("items", StringType(), True)
])

# Load the dataset
df = spark.read.csv("/workspaces/high-performance-pyspark-advanced-strategies-for-optimal-data-processing-3919191/data/online_sales_data.csv", schema=schema, header=True)

# Display the dataset
print("Raw Dataset:")
df.show(10, truncate=False)

In [ ]:
df.describe().show()

In [ ]:
# Find all the NULLs in the dataframe

null_values_count = df.select([count(when(isnull(c) , c)).alias(c) for c in df.columns])

null_values_count.show()

In [ ]:
df.printSchema()

In [ ]:
# Detect rows with negative quantity or invalid price
df_invalid = df.filter((col("quantity") < 0) | (col("quantity").rlike("^[^0-9]")) | (col("price_per_unit").rlike("^[^0-9]")))
df_invalid.show()

In [ ]:
# Data is skewed in qunatity column. So we will impute median for NULLs in this column
# "quantity" is String type, first we will cast it to Numeric and then impute the Median
df = df.withColumn("quantity" , col("quantity").cast("double"))
# Step 1: Calculate the median of the 'quantity' column
median_quantity = df.approxQuantile("quantity", [0.5], 0.0)[0]  # 0.5 for median (50th percentile)
df = df.withColumn("quantity" , when(col("quantity").isNull() , median_quantity).otherwise(col("quantity")))
df = df.withColumn("price_per_unit" , when(col("price_per_unit") == "fifty", 50.00 ).otherwise(col("price_per_unit")))
df = df.withColumn("quantity" , when(col("quantity") < 0 , 0).otherwise(col("quantity")))
df.show(5)

In [ ]:
df = df.withColumn("Customer Name", split(col("customer_details") , "\\|")[0]).\
         withColumn("Customer Address" , split(col("customer_details"), "\\|")[1])

df = df.withColumn("Customer Address" , when(col("Customer Address").isNull() , lit("Unknown")).otherwise(col("Customer Address")))
df.show()

In [ ]:
from pyspark.sql.functions import regexp_extract

# Example of extracting components from customer_details (e.g., street name, city)
df_address_split = df.withColumn('street', regexp_extract(col('customer_details'), r'(\d+ Street Name)', 1)) \
                     .withColumn('city', regexp_extract(col('customer_details'), r'City (\d+)', 1))
df_address_split.show()


In [ ]:
# Data is skewed in quantity column. 
# So we will impute median for NULLs in this column
# "quantity" is String type, first we will cast it to Numeric 
# Then impute the Median
df = df.withColumn("quantity" , col("quantity").cast("double"))

# Step 1: Calculate the median of the 'quantity' column

median_quantity = df.approxQuantile("quantity", [0.5], 0.0)[0]  # 0.5 for median (50th percentile)
df = df.withColumn("quantity" , when(col("quantity").isNull() , median_quantity).otherwise(col("quantity")))
df.show(5)

In [ ]:
# Stop Spark Session
spark.stop()